In [1]:
import pandas as pd
import urllib
import tempfile
import shutil
import zipfile
import os

os.chdir('/Users/joesh/neural_colab_filtering/')
archive_url = f'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
wd = os.getcwd()

with urllib.request.urlopen(archive_url) as url:
    with tempfile.NamedTemporaryFile(delete=True) as f:
        shutil.copyfileobj(url, f)
        with zipfile.ZipFile(f.name) as archive:
            archive.extractall(f'{wd}/datasets')



In [2]:
ratings = pd.read_csv('datasets/ml-1m/ratings.dat', sep='::', engine = 'python', 
                        header=None,
                        names=['user_id', 'movie_id', 'rating', 'timestamp'])

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
import source.model as model
import source.data as data
import torch
import torch.nn as nn

# Create a dataset object

dataset = data.filmDataset( ratings['user_id'].values, ratings['movie_id'].values, 
                            ratings['rating'].values)

mod = model.NCF(ratings.user_id.nunique(), ratings.movie_id.nunique())

In [4]:
# Create a dataloader object

from torch.utils.data import DataLoader
train_set_size = int(0.9 * len(dataset))
test_set_size = len(dataset) - train_set_size

trainset, testset = torch.utils.data.random_split(dataset, [train_set_size, test_set_size])

train_set_size = int(0.8 * len(trainset))
valid_set_size = len(trainset) - train_set_size

train_set, valid_set = torch.utils.data.random_split(trainset, 
                       [train_set_size, valid_set_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last= False)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False, drop_last=False)
test_loader = DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)

# Create a loss function and an optimizer
loss_module = nn.MSELoss()
optimizer = torch.optim.Adam(mod.parameters(), lr=0.01)

In [5]:
d